# Usage of `src.models.fuel` module.

This notebook outlies the basic usage of the `src.models.fuel` module. Used to estimate the fuel usage of a given flight path
 
**Requirements**
 - A csv the flight's state vectors

**Helpful Links**


## Basic Setup

In [ ]:
import sys
# This variable should indicate the path from this Jupyter Notebook to the root directory of the repo.
root_path = '../'
# Adds the repo's root to the list of paths
sys.path.append(root_path)

import sqlite3
import os
import pandas as pd

# Fuel Module
from src.models import fuel

# File containing some flight database
flights_database = os.path.normpath('../data/flight/KDEN_KSEA_2023-01-01_2023-01-31.sqlite')

# Creates the connection to the database
flights_connection = sqlite3.connect(flights_database)

# Reads the first flight_id and corresponding icao24
flight_id, icao24 = pd.read_sql_query("SELECT flight_id, icao24 FROM flights LIMIT 1;", flights_connection).values[0,:]

# Reads that flight's state vectors
state_vectors = pd.read_sql_query(f"""
                                    SELECT DISTINCT state_vectors.*
                                    FROM state_vectors
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}";
                                   """, flights_connection)
# Closes connection
flights_connection.close()

# Computes estimated mass, used fuel, and fuel flow at each point in time
state_vectors = fuel.compute_fuel_data(state_vectors, icao24)

state_vectors

## Computing fuel quantities for all routes and flights

In [ ]:
import sys
# This variable should indicate the path from this Jupyter Notebook to the root directory of the repo.
root_path = '../'
# Adds the repo's root to the list of paths
sys.path.append(root_path)

import sqlite3
import os
import pandas as pd
from IPython.display import clear_output

# Fuel Module
from src.models import fuel

files = ['../data/flight/' + f for f in os.listdir('../data/flight/') if f.endswith('.sqlite')]

new_columns = ['mass', 'fuelflow', 'used_fuel']

i = 0
for file in files:
    i += 1
    clear_output(wait=True)
    print(f'{i}/{len(files)}', file)
    conn = sqlite3.connect(file)
    cursor = conn.cursor()
    
    cursor.execute("DROP TABLE IF EXISTS state_vector_fuel;")

    # Create the new table
    create_table_query = f'''
        CREATE TABLE state_vector_fuel (
            vector_id INTEGER PRIMARY KEY,
            {", ".join([f"{col} REAL" for col in new_columns])}
        );
    '''
    # Create the new table if it doesn't exist
    cursor.execute(create_table_query) 
    
    flight_ids, icao24s = pd.read_sql_query("SELECT flight_id, icao24 FROM flights;", conn).values.T
    
    j = 0
    for flight_id, icao24 in zip(flight_ids, icao24s):
        j += 1
        # Reads that flight's state vectors
        state_vectors = pd.read_sql_query(f"""
                                    SELECT DISTINCT state_vectors.*
                                    FROM state_vectors
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}";
                                   """, conn)
        state_vectors = fuel.compute_fuel_data(state_vectors, icao24)
        print(flight_id, state_vectors.iloc[-1]['used_fuel'], f'{j}/{len(flight_ids)}')
        for index, row in state_vectors.iterrows():
            # Preparing the data to be inserted
            insert_data = tuple(row[col] for col in ['vector_id'] + new_columns)

            # Creating query to insert new values
            insert_query = f'''
                INSERT INTO state_vector_fuel (vector_id, {', '.join(new_columns)})
                VALUES ({', '.join('?' * len(insert_data))})
                ON CONFLICT(vector_id) DO UPDATE SET
                {', '.join([f"{col} = excluded.{col}" for col in new_columns])};
            '''
            cursor.execute(insert_query, insert_data)
        # Commiting changes to the database
        conn.commit()
    conn.close()

In [ ]:
state_vectors